# Coursera Capstone Project - Battle of the Neighborhoods
**Author:** Charles Albert Jester III  
**Date Updated:** 6/1/2020

## 1.	Introduction
### 1.1	Background

When visiting or moving to a new city there can be difficulties when trying to choose or discover restaurants, stores and other local venues to explore in the neighborhoods. This can be daunting for both a user and business perspective when venturing forth into new territory.

When exploring new neighborhoods, there is a difficult task of analyzing various forms of criteria that can drastic impact your decision making. Such factors include, distance, price tier, ratings, photos and tips which is a lot of criteria too look through. 

The Philadelphia area has many universities and colleges make it a top study destination, as the city has evolved into an educational and economic hub, with an estimated gross metropolitan product of $490 billion in 2019. This makes the twelve neighborhoods of Philadelphia a hot spot for new residents, travelers and businesses. The twelve neighborhoods of Philadelphia are Center City, South Philadelphia, Southwest Philadelphia, West Philadelphia, Lower North Philadelphia, Upper North Philadelphia, Bridesburg-Kensington-Port Richmond, Roxborough-Manayunk, Germantown-Chestnut Hill, Olney-Oak Lane, Near Northeast Philadelphia and Far Northeast Philadelphia.

Toronto is an international center for business, finance, arts and culture in the provincial capital of Ontario. Toronto is also an education and economic hub with diversified strengths in technology, design, food services, education and many more. This makes the six neighborhoods a hot spot for new residents, travelers and businesses much like Philadelphia. The six neighborhoods that make up Toronto are Old City of Toronto, East York, Etobicoke, Scarborough, York and North York.


### 1.2 Problem
With the utilization of Foursquare venue data we can utilize this to explore and compare the city of Philadelphia and Toronto. This project will focus on how this data can be used to find new insights required for business opportunities in certain neighborhoods, cluster and segment venues to provide for a better user experience. We can then also compare and distinguish which city venues are better.

By segmenting and clustering venues based on our Foursquare venue data, we can compare and distinguish venue similarities to observe which neighborhoods would be good business ventures. Also, we will utilize factors such as user likes, ratings, tips, photos and distance to venues to seek if there are correlations that exist.

### 1.3 Interest
Interested parties might include business start-ups, stakeholders, and existing owners. Business start-ups might be looking for areas that lack certain categorical venues to further enhance a neighborhoods overall experience. Stakeholders would also be interested in this and also as a means to see how their invested interests are performing as a whole comparatively within city neighborhoods and to another city. Existing business will also benefit to see how the venues likes, tips and photos will bring effectiveness on their price tie and ratings.

## 2. Data Acquisition and Cleaning
### 2.1 Data Sources

Our two primary sources of data will be location information provided from data scrapped from Wikipedia on the neighborhood information for Philadelphia and Toronto plugged into python’s geocoder library. Foursquare will provide our venue details to help explore, segment and analyze the neighborhoods of Philadelphia and Toronto. Below are the links of the Wikipedia sources for location data that was utilized in conjunction with Python’s geocoder:

Philadelphia: https://en.wikipedia.org/wiki/List_of_Philadelphia_neighborhoods

Toronto:  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


## 2.2 Data cleaning

### Loading libraries to extract, explore, segment and cluster information about neighborhoods in Philadelphia and  Toronto.

In [213]:
import numpy as np #Library required for vectorized data handling
import pandas as pd #Library required for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #Library required to handle JSON files

from bs4 import BeautifulSoup #utilized for web scraping data

#!conda install -c conda-forge geopy=1.49.0 --yes # uncomment this line if you haven't installed the following library for python
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge geocoder --yes
import geocoder

import requests # library to handle requests
import urllib.request

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the following library for python
import folium # map rendering library

print('All libraries are imported!')

All libraries are imported!


**Obtain and visualize the underlying HTML code within our webpage**

In [214]:
#Here we establish the URL that gives us the pertinent information on the Neighborhoods an 
urlT = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
urlP = 'https://en.wikipedia.org/wiki/List_of_Philadelphia_neighborhoods'

pageT = urllib.request.urlopen(urlT)
pageP = urllib.request.urlopen(urlP)

#parse the HTML data into the BeautifulSoup parse tree format and visualize using prettify
soupT = BeautifulSoup(pageT, "html.parser")
soupP = BeautifulSoup(pageP, "html.parser")


#print(soupT.prettify())
#print(soupP.prettify())


**Extract the borough and neigborhood information from our underlying HTML code that contains the data important to us for both Toronto and Philadelphia**

In [215]:
#Established the specific table we are looking for within the parsed html for our web page
tablesT = soupT.find("table", class_='wikitable sortable')
tablesP = soupP.findAll("a")
#tablesT
#tablesP

## **Now that we have the raw html output, let's begin cleaning the Philadelphia & Toronto data.**

**Cleaning Philadelphia Data**

In [216]:
#Here we grab the majority of the information that is required, plus a bit of excess. Unfortunately this was the cleanest data I could find on Philadelphia.
PNL = []
for row in soupP.findAll("a"):
    PNL.append(row.get('title'))
    
#print(PNL)

In [217]:
#Here we take a rough outline and filter out the data that is not needed.
dfPRough = pd.DataFrame(PNL, columns=['Neighborhood'])
dfPRough = dfPRough.dropna()
dfPRough = dfPRough.reset_index(drop=True)
dfPRough = dfPRough[8:238]
l2drop=['SEPTA', 'Delaware River', 'Schuylkill River', 'Media/Elwyn Line', 'Semi-detached house', 'Bucks County', 'American Middle class', 'Whites', 'White flight', 'Hispanic and Latino Americans', 'African American', 'Asian American', 'Irish-American',  'Enlarge']
dfPR = dfPRough[~dfPRough.Neighborhood.isin(l2drop)]

In [218]:
#These are to clean up the values for the neighborhoods in the cells so that don't contain redundant values
dfPRtemp = dfPR['Neighborhood'].str.replace('\(page does not exist\)','')
dfPRtemp = dfPRtemp.str.replace('(', "")
dfPRtemp = dfPRtemp.str.replace(')', "")
dfPRtemp = dfPRtemp.str.replace(', Philadelphia, Pennsylvania', "")
dfPRtemp = dfPRtemp.str.replace(', Philadelphia', "")
dfPRtemp = dfPRtemp.str.replace(', PA', "")
dfPRTemp = dfPRtemp.str.replace(', Pennsylvania', "")
dfPR = dfPRtemp.to_frame()

In [219]:
#We sort through finding our boroughs through the use of the Edit Section flag that was leftover. Removing the string
#to set as our borough variable. This will set up our dataframe for finalization
PB = []
PN = []

for row in dfPR['Neighborhood']:
    if "Edit" in row:
        borough = row.replace('Edit section: ', '')
        PB.append(borough)
        PN.append(None)
    else:
        PB.append(borough)
        PN.append(row)

In [220]:
#Finally this gets rid of any None values and ensures that the Borough is not defined as a neighborhood as well.
dfPNe=pd.DataFrame(PB,columns=['Borough'])
dfPNe['Neighborhood'] = PN
dfPNe.dropna(inplace=True)
borP = dfPNe['Borough'].unique()
dfPNe = dfPNe[~dfPNe.Neighborhood.isin(borP)]
dfPNe.head()

,Borough,Neighborhood
2,Center City,Avenue of the Arts Philadelphia
3,Center City,Callowhill
4,Center City,Chinatown
5,Center City,Elfreth's Alley
6,Center City,French Quarter


In [221]:
#Here we observe how many neighborhoods make up each borough and their overall percent of makeup 
#for Philadelphia as a whole.
stnP = dfPNe.Borough
countsP = stnP.value_counts(ascending=False)
percent100P = stnP.value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
tn_summaryP=pd.DataFrame({'Neighborhood': countsP, '%-overall': percent100P})
tn_summaryP

,Neighborhood,%-overall
South Philadelphia,29,16.02%
West Philadelphia,29,16.02%
Center City,21,11.6%
Near Northeast Philadelphia,19,10.5%
Far Northeast Philadelphia,16,8.84%
Southwest Philadelphia,15,8.29%
Lower North Philadelphia,12,6.63%
Germantown-Chestnut Hill,11,6.08%
Olney-Oak Lane,11,6.08%
Upper North Philadelphia,6,3.31%


In [222]:
dfP = pd.DataFrame(borP, columns=['Borough'])
dfP

,Borough
0,Center City
1,South Philadelphia
2,Southwest Philadelphia
3,West Philadelphia
4,Lower North Philadelphia
5,Upper North Philadelphia
6,Bridesburg-Kensington-Port Richmond
7,Roxborough-Manayunk
8,Germantown-Chestnut Hill
9,Olney-Oak Lane


**Cleaning Toronto data**

In [223]:
#We define 3 empty list to be established with our 3 columns from our web page
#these columns are Postal Code, Borough and Neighbourhood.

A=[]
B=[]
C=[]

#Creates a loop to find all rows that begin with <tr> and contain at least 3 columns. If both
#stipulations are met we extract the text values within <td> for each element in that row and
#append them to our lists.
for row in tablesT.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).replace("\n",""))
        B.append(cells[1].find(text=True).replace("\n",""))
        C.append(cells[2].find(text=True).replace("\n",""))

In [224]:
#Here we simply define the column headers and apply their respective lists to fill the value below
dfTNe=pd.DataFrame(A,columns=['Postal Code'])
dfTNe['Borough']=B
dfTNe['Neighborhood']=C

#Finally we exclude any Borough that has a value of 'Not assigned'
dfTNe.drop(dfTNe[dfTNe['Borough'] == 'Not assigned'].index, inplace=True)
dfTNe = dfTNe.reset_index(drop=True)
borT = dfTNe['Borough'].unique()
dfTNe.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [225]:
#Here we observe how many neighborhoods make up each borough and their overall percent of makeup for Toronto as a whole
stnT = dfTNe.Borough
countsT = stnT.value_counts(ascending=False)
percent100T = stnT.value_counts(normalize=True).mul(100).round(2).astype(str) + '%'
tn_summaryT=pd.DataFrame({'Neighborhood': countsT, '%-overall': percent100T})
tn_summaryT

,Neighborhood,%-overall
North York,24,23.3%
Downtown Toronto,19,18.45%
Scarborough,17,16.5%
Etobicoke,12,11.65%
Central Toronto,9,8.74%
West Toronto,6,5.83%
East York,5,4.85%
East Toronto,5,4.85%
York,5,4.85%
Mississauga,1,0.97%


In [226]:
dfT = pd.DataFrame(borT, columns=['Borough'])
dfT

,Borough
0,North York
1,Downtown Toronto
2,Etobicoke
3,Scarborough
4,East York
5,York
6,East Toronto
7,West Toronto
8,Central Toronto
9,Mississauga


**Create and Clean the dataframe to best suit our needs**

**Now that the Philadelphia data has been compiled we shall compile the Toronto data as well.**

**Here I set up the neighborhoods to be search through geocoder to find the latitude and longtiude coordinates for each respective neighborhood.** 

In [227]:
#Repeat a similar process to extracting our data from wikipedia for utilizing
#geocoder to generate latitude and longitude coordinates from the postal codes.

DT=[]
ET=[]
for bor in dfT['Borough']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(bor))
    lat_lng_coords = g.latlng
    
    lat = lat_lng_coords[0]
    lng = lat_lng_coords[1]
    DT.append(lat)
    ET.append(lng)

#output the list of corresponding latitude and longitude to new columns in the dataframe.
dfT['Latitude']=DT
dfT['Longitude']=ET
dfT

,Borough,Latitude,Longitude
0,North York,43.768260,-79.412630
1,Downtown Toronto,43.658200,-79.368320
2,Etobicoke,43.644360,-79.567130
3,Scarborough,43.772200,-79.256660
4,East York,43.691800,-79.327030
5,York,43.692080,-79.478630
6,East Toronto,43.659030,-79.349010
7,West Toronto,43.664712,-79.346346
8,Central Toronto,43.609727,-79.492844
9,Mississauga,43.587260,-79.644940


In [228]:
#Repeat a similar process to extracting our data from wikipedia for utilizing
#geocoder to generate latitude and longitude coordinates from borough information.

DP=[]
EP=[]
for bor in zip(dfP['Borough']):
    g = geocoder.arcgis('{}, Philadelphia, Pennsylvania'.format(bor))
    lat_lng_coords = g.latlng
    
    lat = lat_lng_coords[0]
    lng = lat_lng_coords[1]
    DP.append(lat)
    EP.append(lng)
    
#output the list of corresponding latitude and longitude to new columns in the dataframe.
dfP['Latitude']=DP
dfP['Longitude']=EP
dfP

,Borough,Latitude,Longitude
0,Center City,39.952544,-75.165219
1,South Philadelphia,39.964110,-75.161050
2,Southwest Philadelphia,39.910040,-75.186370
3,West Philadelphia,40.053132,-75.028511
4,Lower North Philadelphia,39.964158,-75.198802
5,Upper North Philadelphia,40.059110,-75.052180
6,Bridesburg-Kensington-Port Richmond,39.980900,-75.099600
7,Roxborough-Manayunk,40.037990,-75.223080
8,Germantown-Chestnut Hill,40.078489,-75.211934
9,Olney-Oak Lane,40.041130,-75.124050


**Here I now add the top 25 venues from each neighborhood**

In [246]:
import os

CLIENT_ID =  os.environ.get("CLIENT_ID")# your Foursquare ID
CLIENT_SECRET =  os.environ.get("CLIENT_SECRET") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")

print('Your credentails:')
print('CLIENT_ID: Are contained within your conda environment variables!' )
print('CLIENT_SECRET: Are contained within your conda environment variables!')

Your credentails:
CLIENT_ID: Are contained within your conda environment variables!
CLIENT_SECRET: Are contained within your conda environment variables!


In [230]:
limit=25

def getNearbyVenues(names, latitude, longitude, radius = 500) :
    venues_list = []
    
    for name, lat, long in zip(names, latitude, longitude):
        print(name)
        
        #Generate the API request url
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION,
            lat,
            long,
            radius,
            limit)
        
        #Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information on nearby venues
        venues_list.append([(
            name,
            lat,
            long,
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['distance'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue ID',
                  'Distance',
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    return(nearby_venues)          

In [231]:
toronto_venues = getNearbyVenues(names=dfT['Borough'],
                                latitude=dfT['Latitude'],
                                longitude=dfT['Longitude']
                                )

North York
Downtown Toronto
Etobicoke
Scarborough
East York
York
East Toronto
West Toronto
Central Toronto
Mississauga


In [242]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.76826,-79.41263,The Keg,5a35b4443abcaf37eb1a0d88,191,43.766579,-79.412131,Steakhouse
1,North York,43.76826,-79.41263,Konjiki Ramen,5a02789d0a464d3112a58785,144,43.766998,-79.412222,Ramen Restaurant
2,North York,43.76826,-79.41263,Toronto Centre for the Arts,4ad4c062f964a520c3f720e3,255,43.766228,-79.414115,Theater
3,North York,43.76826,-79.41263,Loblaws,4ae257cff964a520758d21e3,66,43.768722,-79.412101,Grocery Store
4,North York,43.76826,-79.41263,Satay Sate,57f92db0498ee70159702002,179,43.766690,-79.412100,Indonesian Restaurant


In [233]:
toronto_venues.to_csv('toronto_venues.csv', index=False)

In [234]:
tDataVen = pd.read_csv('toronto_venues.csv')
tDataVen.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.76826,-79.41263,The Keg,5a35b4443abcaf37eb1a0d88,191,43.766579,-79.412131,Steakhouse
1,North York,43.76826,-79.41263,Konjiki Ramen,5a02789d0a464d3112a58785,144,43.766998,-79.412222,Ramen Restaurant
2,North York,43.76826,-79.41263,Toronto Centre for the Arts,4ad4c062f964a520c3f720e3,255,43.766228,-79.414115,Theater
3,North York,43.76826,-79.41263,Loblaws,4ae257cff964a520758d21e3,66,43.768722,-79.412101,Grocery Store
4,North York,43.76826,-79.41263,Satay Sate,57f92db0498ee70159702002,179,43.766690,-79.412100,Indonesian Restaurant


In [235]:
tDataVen.shape

(180, 9)

In [236]:
philadelphia_venues = getNearbyVenues(names=dfP['Borough'],
                                latitude=dfP['Latitude'],
                                longitude=dfP['Longitude']
                                )

Center City
South Philadelphia
Southwest Philadelphia
West Philadelphia
Lower North Philadelphia
Upper North Philadelphia
Bridesburg-Kensington-Port Richmond
Roxborough-Manayunk
Germantown-Chestnut Hill
Olney-Oak Lane
Near Northeast Philadelphia
Far Northeast Philadelphia


In [243]:
philadelphia_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category
0,Center City,39.952544,-75.165219,Dilworth Park,4bde0d566198c9b6c5cc12ff,49,39.952772,-75.164723,Park
1,Center City,39.952544,-75.165219,Philadelphia Film Center,47bdd66df964a520da4d1fe3,195,39.950835,-75.164683,Movie Theater
2,Center City,39.952544,-75.165219,City Hall Courtyard,4f29e8e3e4b02f0aff55b2a7,138,39.952484,-75.163592,Plaza
3,Center City,39.952544,-75.165219,Del Frisco's Double Eagle Steak House,4ab2ac0bf964a520d66b20e3,177,39.950956,-75.165459,Steakhouse
4,Center City,39.952544,-75.165219,"The Ritz-Carlton, Philadelphia",4a68db6ff964a52023cb1fe3,152,39.951446,-75.164149,Hotel


In [238]:
philadelphia_venues.to_csv("philly_venues.csv", index=False)

In [239]:
pDataVen = pd.read_csv('philly_venues.csv')
pDataVen.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category
0,Center City,39.952543,-75.165219,Dilworth Park,4bde0d566198c9b6c5cc12ff,49,39.952772,-75.164723,Park
1,Center City,39.952543,-75.165219,Philadelphia Film Center,47bdd66df964a520da4d1fe3,195,39.950835,-75.164683,Movie Theater
2,Center City,39.952543,-75.165219,City Hall Courtyard,4f29e8e3e4b02f0aff55b2a7,138,39.952484,-75.163592,Plaza
3,Center City,39.952543,-75.165219,Del Frisco's Double Eagle Steak House,4ab2ac0bf964a520d66b20e3,177,39.950956,-75.165459,Steakhouse
4,Center City,39.952543,-75.165219,"The Ritz-Carlton, Philadelphia",4a68db6ff964a52023cb1fe3,152,39.951446,-75.164149,Hotel


In [240]:
pDataVen.shape

(268, 9)

## Now that the location information is compiled,  we must work on adding in the details of our venues. Let's create a function.

In [244]:
CLIENT_ID="ELS3CERVGDUA1T34ZIMIERDX2A344AHEOJUMKDATY4TMC1P1"
CLIENT_SECRET="A1MK2X0RPT1KJB1XJO3I5THLGU50YOIM4CYSV1MJK5NQCNPX"
ACCESS_TOKEN="1XY5Z2BP2SPSSJPLTEBORYRQ4B3IGVCDYKMFT2OH4F5AUA02"

#vID = '4b2c4178f964a520b3c424e3'
#url='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            #vID,
            #CLIENT_ID,
            #CLIENT_SECRET,
            #ACCESS_TOKEN,
            #VERSION)
#results = requests.get(url).json()['response']['venue']
#results

In [247]:
#here we define the getVenuesDetails function to find information regarding ratings, likes counts, pic counts,
#tips counts and reasons count
def getVenuesDetails(names, vIDs) :
    dfname=[]
    dflc=[]
    dfra=[]
    dfpc=[]
    dfrc=[]
    dftc=[]
    
    #Utilizes a for loop to grab details of each venue 
    for name, vID in zip(names, vIDs):
       
        print(name)
        
        #Generate the API request url
        url='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            vID,
            CLIENT_ID,
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION)
        
        #Make the GET request
        results = requests.get(url).json()["response"]['venue']
        
        #Ensures the venue name is included in the output
        dfname.append(name)
        
        #return only relevant information on venues, if that information results in a KeyError
        #return a Null to the output to make cleaning easier
        try:
            dflc.append(results['likes']['count'])
        except KeyError:
            dflc.append('Null')
            pass
        try:
            dfra.append(results['rating'])
        except KeyError:
            dfra.append('Null')
            pass
        try:
            dfpc.append(results['photos']['count'])
        except KeyError:
            dfpc.append('Null')
            pass
        try: 
            dfrc.append(results['reasons']['count'])
        except KeyError:
            dfrc.append('Null')
            pass
        try:
            dftc.append(results['tips']['count'])
        except KeyError:
            dftc.append('Null')
            pass
        
        #Pass data extracted from the json information into a new dataframe with the approriate column headers.
        test = list(zip(dfname,dflc,dfra,dfpc,dfrc,dftc))
        dfFinal = pd.DataFrame(test, columns=['Venue',
                                              'like_counts', 
                                              'rating', 
                                              'photo_count', 
                                              'reasons_count', 
                                              'tips_count'])
  
   

    return(dfFinal)

## Testing function and post-query merged data
**NOTE: Testing on small sub-sample to ensure the premium calls are working appropriately. Otherwise, it can easily surpass the 500 daily call limit on your Foursquare account.**

In [248]:
#Here I'm creating a simple dataframe to test my function on prior to calling it on all of our data.
#I devised this as I'm using premium calls for the venue ratings, picture counts, tip counts and so on.
l = ['Dilworth Park', 'SEPTA BS', 'Philadelphia Film Center']
v = ['4bde0d566198c9b6c5cc12ff', '4b2c4178f964a520b3c424e3', '47bdd66df964a520da4d1fe3']

#Constructs a small dataframe for testing, using output with known "Null" results
dfl = pd.DataFrame(l, columns=['Name'])
dfl['vID']=v

#
dfl

,Name,vID
0,Dilworth Park,4bde0d566198c9b6c5cc12ff
1,SEPTA BS,4b2c4178f964a520b3c424e3
2,Philadelphia Film Center,47bdd66df964a520da4d1fe3


In [249]:
#Run the above dataframe set with our function
venueTest = getVenuesDetails(names=dfl['Name'], vIDs=dfl['vID'])


Dilworth Park


KeyError: 'venue'

In [250]:
#observe the output
venueTest

,Venue,like_counts,rating,photo_count,reasons_count,tips_count
0,Dilworth Park,311,9.1,668,1,23
1,SEPTA BS,7,Null,22,0,4
2,Philadelphia Film Center,88,8.9,177,1,20


In [251]:
#Here we run an innermerge which will remove any venue where the 'rating' resulted in 'Null'
#This gives us our example output and proves to us the function is working properly.
mergedPtest = pDataVen.merge(venueTest, how = 'inner', on = ['Venue'])
mergedPtest

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category,like_counts,rating,photo_count,reasons_count,tips_count
0,Center City,39.952543,-75.165219,Dilworth Park,4bde0d566198c9b6c5cc12ff,49,39.952772,-75.164723,Park,311,9.1,668,1,23
1,Center City,39.952543,-75.165219,Philadelphia Film Center,47bdd66df964a520da4d1fe3,195,39.950835,-75.164683,Movie Theater,88,8.9,177,1,20


## Finalizing our dataset by merging our information together for each city.

In [176]:
#first lets start with toronto as it has a smaller number of venues overall, just incase we run into any problems
#we can still potentially not reach our quota for the day.
venueDetails_toronto = getVenuesDetails(names=toronto_venues['Venue'],
                                       vIDs=toronto_venues['Venue ID'])

Soho House Toronto


KeyError: 'venue'

In [139]:
#Save the output to a csv, this is crucial so we won't have to run the above premium call again.
venueDetails_toronto.to_csv("toronto_venueDets.csv", index=False)

In [150]:
#Read the csv file back in as a dataframe to be merged with our borough dataframe.
toronto_vendets = pd.read_csv("toronto_venueDets.csv")
toronto_vendets.shape

(129, 5)

In [141]:
#Next, we will run the same query function on our philadelphia dataset
venueDetails_philadelphia = getVenuesDetails(names=philadelphia_venues['Venue'], 
                                             vIDs=philadelphia_venues['Venue ID'])

Dilworth Park
Philadelphia Film Center
City Hall Courtyard
Del Frisco's Double Eagle Steak House
The Ritz-Carlton, Philadelphia
JFK Plaza / Love Park
La Colombe Coffee Roasters
sweetgreen
Rothman Institute Ice Rink at Dilworth Park
The Wanamaker Organ
One Liberty Observation Deck
Oyster House
The Capital Grille
UNIQLO
Fogo De Chão
Mission Taqueria
HipCityVeg
Five Below
R2L
Snap Custom Pizza
Blick Art Materials
Bikram Yoga of Philadelphia
Anthony's O2 Skincare Lounge
Insomnia Cookies
City Hall Tower
Osteria
Santucci's Original Square Pizza
South Restaurant
Jimmy G's Steaks
Stockyard
John's Place
Prohibition Tap Room
Cafe Lift
performance garage
The Institute Bar
Retro Fitness - Spring Garden
Kelliann's Bar & Grill
Brandywine Pizza
Lorraine
City View Pizza
Sals Seafood
Divine Lorraine Hotel
Cvs
Retro Fitness
SUBWAY
Sakura Japanese Cuisine
Thirsty Dice
Anytime Fitness
Johnnie Bleu
Saint Stephen's Green
Dunkin'
Penrose Diner
Popi's Restaurant
Subway
APlus at Sunoco
Peking Inn
Packer Park L

In [142]:
#Saving the ouput to a csv yet again to prevent the need to resubmit the premium calls
venueDetails_philadelphia.to_csv("philly_venueDets.csv", index=False)

In [184]:
#Read the information back into a dataframe to be merged which the borough dataframe.
philadelphia_vendets = pd.read_csv("toronto_venueDets.csv")
philadelphia_vendets.head()

,like_counts,rating,photo_count,reasons_count,tips_count
0,221,8.8,179,1,30
1,222,8.6,454,1,45
2,29,8.4,27,0,6
3,181,8.4,517,1,47
4,36,8.3,45,1,13


## Final Clean up

In [146]:
#Here we run an inner merge which will remove any venue where the 'rating' resulted in 'Null'
#This gives us our final output that we can be to run tests on.
mergedP = pDataVen.merge(philadelphia_vendets, how = 'inner', on = ['Venue'])
mergedT = tDataVen.merge(toronto_vendets, how = 'inner', on = ['Venue'])

,like_counts,rating,photo_count,reasons_count,tips_count
0,221,8.8,179,1,30
1,222,8.6,454,1,45
2,29,8.4,27,0,6
3,181,8.4,517,1,47
4,36,8.3,45,1,13
5,270,8.3,259,1,81
6,243,9.0,158,1,85
7,686,8.8,467,1,206
8,318,8.4,833,1,42
9,43,8.2,69,1,18


## Let's have a look at each of the final dataframes that we will be using for our data analysis stage

In [ ]:
#First, let's observe the results of our merged data set for philadelphia
mergedT.head()

In [160]:
#Finally, let's observe the results of our merged data set for philadelphia
mergedP.head()

Dilworth Park
SEPTA BS
Philadelphia Film Center


## 2.3 Data Limitations
Currently, the two limiting factors in a more robust data set is the fact that we have to limit the API calls for premium data, as a foursquare personal account restricts the premium call daily limit to 500. Thus, we look at the boroughs overall and only the top 25 venues in each borough. We are also limited by the amount of available data provided by foursquare at each of these venues, as they can be missing important information pertaining to our regression analyses. As such, this is more a modeling of what is possible with our the queriable data and could be expanded upon when limitations are lifted.

## 2.4 How the Data Will Be Utilized


,Venue,like_counts,rating,photo_count,reasons_count,tips_count
0,Dilworth Park,311,9.1,668,1,23
1,SEPTA BS,7,Null,22,0,4
2,Philadelphia Film Center,88,8.9,177,1,20


In [245]:
vID = '4bde0d566198c9b6c5cc12ff'
url='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            vID,
            CLIENT_ID,
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION)

requests.get(url).json()

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5ed54a540f596800254f36ad'},
 'response': {}}

,Venue,like_counts,rating,photo_count,reasons_count,tips_count,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Distance,Venue Latitude,Venue Longitude,Venue Category
0,Dilworth Park,311,9.1,668,1,23,Center City,39.952543,-75.165219,4bde0d566198c9b6c5cc12ff,49,39.952772,-75.164723,Park
1,Philadelphia Film Center,88,8.9,177,1,20,Center City,39.952543,-75.165219,47bdd66df964a520da4d1fe3,195,39.950835,-75.164683,Movie Theater
